In [1]:
import numpy as np
import matplotlib.pyplot as plt

import random

from matplotlib.colors import ListedColormap
from sklearn import datasets, model_selection

В коде из методички реализуйте один или несколько из критериев останова (количество листьев, количество используемых признаков, глубина дерева и т.д.).

Для задачи классификации обучить дерево решений с использованием критериев разбиения Джини и Энтропия. Сравнить качество классификации, сделать выводы.

In [2]:
# Реализуем класс узла
class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

In [3]:
# Класс терминального узла (листа)
class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()        
        
    def predict(self):
              
        # подсчет количества объектов разных классов
        classes = {}  # сформируем словарь "класс: количество объектов"
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
        #  найдем класс, количество объектов которого будет максимальным в этом листе и вернем его        
        prediction = max(classes, key=classes.get)        
        return prediction    

In [4]:
class Tree:
    
    def __init__(self, n_features, min_leaf, criteria):
        self.n_features = n_features
        self.min_leaf = min_leaf
        self.criteria = criteria
             
        
    def info_criteria(self, labels):
        if self.criteria=='variance': 
            return np.mean((labels - np.mean(labels))**2) 
        #  подсчет количества объектов разных классов
        classes = {}
        for label in labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1

        #  расчет критерия
        entropy_ = 0
        impurity = 1    
        for label in classes:
            p = classes[label] / len(labels)
            entropy_ -= p * np.log2(p)
            impurity -= p ** 2    

        if self.criteria == 'entropy':        
            return entropy_
        elif self.criteria == 'gini':
            return impurity
        
        
    # Расчет качества
    def quality(self, true_labels, false_labels, current_criteria):        
        # доля выбоки, ушедшая в левое поддерево
        p = float(true_labels.shape[0]) / (true_labels.shape[0] + false_labels.shape[0])
        return current_criteria - p * self.info_criteria(true_labels) - (1 - p) * self.info_criteria(false_labels)         
        
        
    # Разбиение датасета в узле
    def split(self, data, labels, index, t):

        left = np.where(data[:, index] <= t)
        right = np.where(data[:, index] > t)

        true_data = data[left]
        false_data = data[right]
        true_labels = labels[left]
        false_labels = labels[right]

        return true_data, false_data, true_labels, false_labels 
        
        
    # Нахождение наилучшего разбиения
    def find_best_split(self, data, labels):

        current_criteria = self.info_criteria(labels)

        best_quality = 0
        best_t = None
        best_index = None
        
        random.seed(42)
        selected_features =  random.sample(range(data.shape[1]), self.n_features)

        for index in selected_features:
            # будем проверять только уникальные значения признака, исключая повторения
            t_values = np.unique([row[index] for row in data])

            for t in t_values:
                true_data, false_data, true_labels, false_labels = self.split(data, labels, index, t)
                
                #  пропускаем разбиения, в которых в узле остается менее 5 объектов
                if len(true_data) < self.min_leaf or len(false_data) < self.min_leaf:
                    continue

                current_quality = self.quality(true_labels, false_labels, current_criteria)

                #  выбираем порог, на котором получается максимальный прирост качества
                if current_quality > best_quality:
                    best_quality, best_t, best_index = current_quality, t, index
                    
        return best_quality, best_t, best_index        
          
        
    # Построение дерева с помощью рекурсивной функции
    def build_tree(self, data, labels): 

        quality, t, index = self.find_best_split(data, labels)        

        #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
        if quality == 0:
            return Leaf(data, labels)

        true_data, false_data, true_labels, false_labels = self.split(data, labels, index, t)        

        # Рекурсивно строим два поддерева
        true_branch = self.build_tree(true_data, true_labels)
        false_branch = self.build_tree(false_data, false_labels)

        # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева        
        return Node(index, t, true_branch, false_branch)
    
    
    def fit(self, data, labels):
        self.my_tree = self.build_tree(data, labels)
        
    
    def classify_object(self, obj, node):
        #  Останавливаем рекурсию, если достигли листа
        if isinstance(node, Leaf):
            answer = node.prediction
            return answer

        if obj[node.index] <= node.t:
            return self.classify_object(obj, node.true_branch)
        else:
            return self.classify_object(obj, node.false_branch)
        
        
    def predict(self, data):        
        classes = []
        for obj in data:
            prediction = self.classify_object(obj, self.my_tree)        
            classes.append(prediction)
        return classes

In [5]:
# Введем функцию подсчета точности как доли правильных ответов
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [6]:
# сгенерируем данные
classification_data, classification_labels = datasets.make_classification(n_features = 12, n_informative = 9, 
                                                      n_classes = 2, n_redundant=0, 
                                                      n_clusters_per_class=1, random_state=42)

In [7]:
# Разобьем выборку на обучающую и тестовую
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(classification_data, 
                                                                                     classification_labels, 
                                                                                     test_size = 0.3,
                                                                                     random_state = 1)

n_features=8, min_leaf=5, criteria='entropy'

In [8]:
tree = Tree(n_features=8, min_leaf=5, criteria='entropy')   #gini
tree.fit(train_data, train_labels)

train_answers = tree.predict(train_data)
test_answers = tree.predict(test_data)

In [9]:
# Точность на обучающей выборке
train_accuracy = accuracy_metric(train_labels, train_answers)
train_accuracy

94.28571428571428

In [10]:
# Точность на тестовой выборке
test_accuracy = accuracy_metric(test_labels, test_answers)
test_accuracy

93.33333333333333

Вывод: Критерии разбиения Джини и Энтропия дают одинаковое (или очень близкое) качество классификации.